# Flood Detection Challenge

This repository contains code for our submission to the **ETCI 2021 Competition on Flood Detection** ([Leaderboard](https://competitions.codalab.org/competitions/30440), [Homepage](https://nasa-impact.github.io/etci2021/)) (Winning
Solution #2). 

Accompanying paper: [Flood Segmentation on Sentinel-1 SAR Imagery with Semi-Supervised Learning](http://arxiv.org/abs/2107.08369).

by Sayak Paul\*, Siddha Ganju\*.

(\*) equal contribution.

<div align="center">
	<img src="https://i.ibb.co/X7chPyT/pipeline.png" width=600/>
</div><br>

## Team 

* [Siddha Ganju](http://sidgan.github.io/siddhaganju)
* [Sayak Paul](https://sayak.dev)

# Code

## Data Preparation
Unzip the data to working directory. 
<br>
**Note**: We're working with a large data set. It could take ~40 mins to unzip all files. 

In [ ]:
!unzip -qq data/train.zip -d ./

In [ ]:
!unzip -qq data/test.zip -d ./

In [ ]:
!unzip -qq data/test_internal.zip -d ./

## Data Exploration
Data exploration is available in `notebooks/Data_Viz.ipynb`. 

Link to the notebook is [**here**](Data_Viz.ipynb)

## Training and Submission

_**⚠️ &nbsp;&nbsp; Make sure the data is downloaded and is placed in the right paths as specified in
`src/config.py`.**_

1. `train.py`: First, we train two models on the training set: UNet [1] and UNet++ [2] with a MobileNetV2 [3] backend.

**Note**: The training process takes a significant amount of time, for the sake of brevity. We're provided you an exemplary trained weights in the `Best_IoU` directory.

PyTorch enables multiprocessing, but we should increase the shared memory (shm) with the below command to make sure it's not limited. 

In [ ]:
!mount -o remount,size=16G /dev/shm

In [ ]:
# The training process could take up to 1.5 hours
# This will generate supervised_training.log and unet_mobilenet_v2_{rank}.pth
%run src/train_unet.py

In [ ]:
# The training process could take up to 1.5 hours
# This will generate supervised_training.log and upp_mobilenet_v2_{rank}.pth
%run src/train_upp.py

**Hint**: Did you get the below error message? Please restart the kernel. 

2. `notebooks/Generate_Pseudo.ipynb`:
    * We then use their averaged ensemble to generate pseudo-labels on the unlabeled test set.
    * We then create a new training set with these generated pseudo-labels and the existing training set.

Link to the notebook is [**here**](Generate_Pseudo.ipynb). The execution of this notebook will generate a `pseudo_labels` directory, `test_sentinel.csv`, and `pseudo_df.csv`. 

3. `train_pseudo_label.py`: We fine-tune the **UNet model weights** obtained from step 1 on this newly created training set. Note that to run this script one _must_ first run the `notebooks/Generate_Pseudo.ipynb` notebook, generate the pseudo-labels and set the dataframe path (`pseudo_df` variable) accordingly inside this script. 

In [ ]:
# The training process could take up to 1 hour
# This will generate pseudo_label.log and unet_pseudo_mobilenetv2_round1_{rank}.pth
%run src/train_pseudo_label.py -p Best_IoU/unet_mobilenet_v2_0.pth -f unet_pseudo_mobilenetv2_round1

4. Repeat for n rounds. In our experiments, we do it for 2 rounds. 
**Note**: After the first round of pseudo-labeling, we refine our pseudo-labels. This process is again
governed by an averaged ensemble but instead of using just two models we also add the model (trained with pseudo-labels
in the last iteration) to the ensemble. This change is incorporated in the `notebooks/Generate_Pseudo.ipynb` notebook
and is executed thereafter.

> This idea of pseudo-label generation is taken from this talk: [How to cook pseudo-labels](https://www.youtube.com/watch?v=SsnWM1xWDu4).

Link to the `Generate_Pseudo.ipynb` notebook is [**here**](Generate_Pseudo.ipynb). The execution of this notebook will generate a `pseudo_label` directory. 

In [ ]:
# The training process could take up to 1 hour
# This will generate pseudo_label.log and unet_pseudo_mobilenetv2_round1_{rank}.pth
%run src/train_pseudo_label.py -p Best_IoU/unet_mobilenet_v2_0.pth -f unet_pseudo_mobilenetv2_round2

After completing training for the final round, we do the following:

* Using the `notebooks/Ensemble_Inference.ipynb` notebook, we first generate the initial predictions. We use stacking for ensembling. 

Link to the notebook is [**here**](Ensemble_Inference.ipynb). 

* We then apply Conditional Random Fields [4] on the predictions to enhance segmentation boundaries. This is demonstrated in the `notebooks/Apply_CRF.ipynb` notebook.

Link to the notebook is [**here**](Apply_CRF.ipynb). 

### Additional notes on our inference pipeline

* For creating the model ensemble, we use the initially trained UNet and UNet++ models along with the _last_
  fine-tuned UNet model from the pseudo-labeling step.
* To further account for uncertainty and improve our predictions, we apply test-time augmentation using the [`ttach`](https://github.com/qubvel/ttach)
library. 
  
In case you have any difficulties understanding the overall workflow feel free to open an issue on GitHub and we will get back to you. 

## References

[1] Ronneberger O., Fischer P., Brox T. (2015) U-Net: Convolutional Networks for Biomedical Image Segmentation. In: Navab N., Hornegger J., Wells W., Frangi A. (eds) Medical Image Computing and Computer-Assisted Intervention – MICCAI 2015. MICCAI 2015. Lecture Notes in Computer Science, vol 9351. Springer, Cham. https://doi.org/10.1007/978-3-319-24574-4_28

[2] Zhou, Z., Siddiquee, M., Tajbakhsh, N., & Liang, J. (2018). UNet++: A Nested U-Net Architecture for Medical Image Segmentation. Deep Learning in Medical Image Analysis and Multimodal Learning for Clinical Decision Support : 4th International Workshop, DLMIA 2018, and 8th International Workshop, ML-CDS 2018, held in conjunction with MICCAI 2018, Granada, Spain, S..., 11045, 3–11. https://doi.org/10.1007/978-3-030-00889-5_1

[3] M. Sandler, A. Howard, M. Zhu, A. Zhmoginov and L. Chen, "MobileNetV2: Inverted Residuals and Linear Bottlenecks," 2018 IEEE/CVF Conference on Computer Vision and Pattern Recognition, 2018, pp. 4510-4520, doi: 10.1109/CVPR.2018.00474.

[4] Philipp Krähenbühl and Vladlen Koltun. 2011. Efficient inference in fully connected CRFs with Gaussian edge potentials. In <i>Proceedings of the 24th International Conference on Neural Information Processing Systems</i> (<i>NIPS'11</i>). Curran Associates Inc., Red Hook, NY, USA, 109–117.

[5] Q. Xie, M. -T. Luong, E. Hovy and Q. V. Le, "Self-Training With Noisy Student Improves ImageNet Classification," 2020 IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR), 2020, pp. 10684-10695, doi: 10.1109/CVPR42600.2020.01070.

[6] Berthelot, David, et al. “AdaMatch: A Unified Approach to Semi-Supervised Learning and Domain Adaptation.” ArXiv:2106.04732 [Cs], June 2021. arXiv.org, http://arxiv.org/abs/2106.04732.

## Acknowledgements

* We are grateful to the [ML-GDE program](https://developers.google.com/programs/experts/) for providing GCP credits to support our experiments. 
* Thanks to [Charmi Chokshi](https://in.linkedin.com/in/charmichokshi), and domain experts Shubhankar Gahlot, May Casterline, Ron Hagensieker, Lucas Kruitwagen, Aranildo Rodrigues, Bertrand Le Saux, Sam Budd, Nick Leach, and, Veda Sunkara for insightful discussions.

## Citation

```
@inproceedings{paul2021flood,
    title   = {Flood Segmentation on Sentinel-1 SAR Imagery with Semi-Supervised Learning},
    author  = {Sayak Paul and Siddha Ganju},
    year    = {2021},
    URL = {https://arxiv.org/abs/2107.08369},
    booktitle = {NeurIPS Tackling Climate Change with Machine Learning Workshop}
}
```